In [1]:
from collections import deque

import gym
import random
import numpy as np
import torch
from torch import nn
import torch.autograd as autograd
import torch.optim as optim
random.seed(0)


def get_screen(env):
    # Returned screen requested by gym is 400x600x3, but is sometimes larger
    # such as 800x1200x3. Transpose it into torch order (CHW).
    return env.render(mode='rgb_array').transpose((2, 0, 1))


class Cnn(nn.Module):

    def __init__(self, classes_count):
        super().__init__()

        self.conv_net = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),  # (b x 96 x 55 x 55)
            # nn.ReLU(),
            nn.LeakyReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),  # section 3.3
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 96 x 27 x 27)
            nn.Conv2d(96, 256, 5, padding=2),  # (b x 256 x 27 x 27)
            # nn.ReLU(),
            nn.LeakyReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 13 x 13)
            nn.Conv2d(256, 384, 3, padding=1),  # (b x 384 x 13 x 13)
            # nn.ReLU(),
            nn.LeakyReLU(),
            nn.Conv2d(384, 384, 3, padding=1),  # (b x 384 x 13 x 13)
            # nn.ReLU(),
            nn.LeakyReLU(),
            nn.Conv2d(384, 256, 3, padding=1),  # (b x 256 x 13 x 13)
            # nn.ReLU(),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 6 x 6)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(in_features=(256 * 5 * 3), out_features=4096),
            # nn.ReLU(),
            nn.LeakyReLU(),
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(in_features=4096, out_features=4096),
            # nn.ReLU(),
            nn.LeakyReLU(),
            nn.Linear(in_features=4096, out_features=classes_count),
        )

        # self.init_bias()

    def init_bias(self):
        for layer in self.conv_net:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.01)
                nn.init.constant_(layer.bias, 0)
        nn.init.constant_(self.conv_net[4].bias, 1)
        nn.init.constant_(self.conv_net[10].bias, 1)
        nn.init.constant_(self.conv_net[12].bias, 1)

    def forward(self, x):
        x = self.conv_net(x)
        x = x.view(-1, 256 * 5 * 3)  # reduce the dimensions for linear layer input
        return self.classifier(x)

class ReplayBuffer:

    def __init__(self, max_size):
        self.max_size = max_size
        self.buffer = deque(maxlen=max_size)

    def add_replay(self, state, action, reward, next_state, done):
        experience = (state, action, np.array([reward]), next_state, done)
        self.buffer.append(experience)

    def sample(self, batch_size):
        state_batch = []
        action_batch = []
        reward_batch = []
        next_state_batch = []
        done_batch = []

        batch = random.sample(self.buffer, batch_size)

        for experience in batch:
            state, action, reward, next_state, done = experience
            state_batch.append(state)
            action_batch.append(action)
            reward_batch.append(reward)
            next_state_batch.append(next_state)
            done_batch.append(done)

        return (state_batch, action_batch, reward_batch, next_state_batch, done_batch)

    def __len__(self):
        return len(self.buffer)

class DQNAgent:

    def __init__(self, actions):
        self.classes = actions
        self.net = Cnn(len(actions))
        self.optimizer = optim.Adam(params=self.net.parameters(), lr=0.0001)
        self.replay_buffer = ReplayBuffer(1000)
        self.gamma = 0.99
        self.epsilon = 0.5
        self.MSE_loss = nn.MSELoss()

    def get_action(self, state):
        state = autograd.Variable(torch.from_numpy(state).float().unsqueeze(0))
        qvals = self.net(state)
        print(qvals.tolist()[0])
        if (random.random() < self.epsilon):
            action = np.argmax(qvals.detach().numpy())
            print("Choosed best action {}".format(action))
        else:
            action = random.randint(0, len(qvals.tolist()[0])-1)
            print("Choosed random action {}".format(action))

        return action

    def compute_loss(self, batch):
        b_states, b_actions, b_rewards, b_next_states, b_dones = batch
        states = torch.FloatTensor(b_states)
        actions = torch.LongTensor(b_actions)
        rewards = torch.FloatTensor(b_rewards)
        next_states = torch.FloatTensor(b_next_states)
        dones = torch.FloatTensor(b_dones)

        curr_Q = self.net.forward(states).gather(1, actions.unsqueeze(1))
        curr_Q = curr_Q.squeeze(1)
        next_Q = self.net.forward(next_states)
        max_next_Q = torch.max(next_Q, 1)[0]
        expected_Q = rewards.squeeze(1) + (1 - dones) * self.gamma * max_next_Q

        loss = self.MSE_loss(curr_Q.clone(), expected_Q.detach())
        return loss

    def train(self, batch_size):

        if (len(self.replay_buffer.buffer) >= batch_size):
            batch = self.replay_buffer.sample(batch_size)
            loss = self.compute_loss(batch)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()


def main():
    env = gym.make('Pong-v0', render_mode="human") #, render_mode="human"
    env.seed(0)
    actions = [i for i in range(0, env.action_space.n)]
    print(actions)

    agent = DQNAgent(actions)

    env.reset()
    current_state = get_screen(env)

    episode_reward = 0
    while True:
        action = agent.get_action(current_state)
        old_state = current_state
        _, reward, done, _ = env.step(action)
        current_state = get_screen(env)
        episode_reward += reward

        agent.replay_buffer.add_replay(old_state, action, reward, current_state, done)
        agent.train(4)

        print('Took action {} and got reward {}'.format(action, reward))

        if done:
            print('Reward: %s' % episode_reward)
            break


if __name__ == '__main__':
    main()

/usr/local/lib/python3.9/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v5` with the environment ID `ALE/Pong-v5`.
  logger.warn(
A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]
/usr/local/lib/python3.9/site-packages/gym/utils/seeding.py:138: DeprecationWarning: WARN: Function `hash_seed(seed, max_bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
/usr/local/lib/python3.9/site-packages/gym/utils/seeding.py:175: DeprecationWarning: WARN: Function `_bigint_from_bytes(bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(


[0, 1, 2, 3, 4, 5]
[-0.052000049501657486, 0.06453297287225723, 0.05515654757618904, -0.05472005158662796, -0.013920728117227554, -0.09888757765293121]
Choosed random action 3
Took action 3 and got reward 0.0
[-0.16122084856033325, 0.038680970668792725, 0.1276588886976242, 0.12168605625629425, -0.0787091851234436, -0.04827336594462395]
Choosed best action 2
Took action 2 and got reward 0.0
[0.057489216327667236, 0.0680910050868988, 0.045492593199014664, 0.10023170709609985, 0.05018559470772743, -0.034426022320985794]
Choosed random action 3


/usr/local/lib/python3.9/site-packages/gym/utils/seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


Took action 3 and got reward 0.0
[-0.05114205181598663, -0.01631111465394497, 0.051099009811878204, 0.016928372904658318, 0.10367612540721893, -0.08670305460691452]
Choosed best action 4


/var/folders/82/c314q3dx0dx676lpzspl34l40000gn/T/ipykernel_85413/2444152256.py:133: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  states = torch.FloatTensor(b_states)


Took action 4 and got reward 0.0
[-0.03672539070248604, 0.037025950849056244, 0.6268243193626404, 1.0260262489318848, 0.45468664169311523, -0.2167251706123352]
Choosed random action 2
Took action 2 and got reward 0.0
[0.0002922937273979187, 0.019849367439746857, 2.298460006713867, 2.297649621963501, 0.7248602509498596, -0.1423492133617401]
Choosed random action 4
Took action 4 and got reward 0.0
[-0.018096603453159332, -0.0061476826667785645, 4.962627410888672, 4.613903522491455, 2.8697640895843506, -0.24116194248199463]
Choosed best action 2
Took action 2 and got reward 0.0
[-0.6102304458618164, 0.3053390383720398, 10.936654090881348, 11.34432601928711, 8.38618278503418, -0.6020138263702393]
Choosed random action 2
Took action 2 and got reward 0.0
[-0.0005893334746360779, 1.824238657951355, 24.12952995300293, 22.392940521240234, 20.629131317138672, -2.093247175216675]
Choosed random action 3
Took action 3 and got reward 0.0
[-1.8795547485351562, -1.7959526777267456, 45.872413635253906

Error: Canceled future for execute_request message before replies were done

In [ ]:
env = gym.make('Pong-v0', render_mode="human") #, render_mode="human"
env.seed(0)
actions = [i for i in range(0, env.action_space.n)]
print(actions)

agent = DQNAgent(actions)

env.reset()
current_state = get_screen(env)

episode_reward = 0
while True:
    action = agent.get_action(current_state)
    old_state = current_state
    _, reward, done, _ = env.step(action)
    current_state = get_screen(env)
    episode_reward += reward

    agent.replay_buffer.add_replay(old_state, action, reward, current_state, done)
    agent.train(4)

    print('Took action {} and got reward {}'.format(action, reward))

    if done:
        print('Reward: %s' % episode_reward)
        break